In [ ]:
import cv2
import numpy as np

# Global variables to store the ROI coordinates
roi_selected = False
roi = (0, 0, 0, 0)

# Mouse callback function to draw a rectangle around the ROI
def select_roi(event, x, y, flags, param):
    global roi, roi_selected, start_point

    if event == cv2.EVENT_LBUTTONDOWN:
        start_point = (x, y)
        roi_selected = False

    elif event == cv2.EVENT_MOUSEMOVE:
        if 'start_point' in globals():
            img_copy = image.copy()
            cv2.rectangle(img_copy, start_point, (x, y), (255, 0, 0), 2)
            cv2.imshow('Select ROI', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        roi = (start_point[0], start_point[1], x - start_point[0], y - start_point[1])
        roi_selected = True

# Load the image
image = cv2.imread('tic_tac_toe_board.jpg')
cv2.namedWindow('Select ROI')
cv2.setMouseCallback('Select ROI', select_roi)

# Show the image and wait for the ROI to be selected
while True:
    cv2.imshow('Select ROI', image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or roi_selected:
        break

cv2.destroyWindow('Select ROI')

# If ROI was not selected, exit
if not roi_selected:
    print("No ROI selected, exiting.")
    exit()

# Extract the ROI
x, y, w, h = roi
roi_image = image[y:y+h, x:x+w]

# Display the selected ROI
cv2.imshow('Selected ROI', roi_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Display coordinates
print(x, y, w, h)


In [ ]:
import cv2
import numpy as np

# Function to do nothing on trackbar callback
def nothing(x):
    pass

# Load the image
image = cv2.imread('tic_tac_toe_board.jpg')

# Predefined ROI parameters
x, y, w, h = 642, 431, 737, 550

# Extract the ROI
roi_image = image[y:y+h, x:x+w]

# Create a window for Canny edge detection
cv2.namedWindow('Canny Edge Detection')
cv2.createTrackbar('Threshold1', 'Canny Edge Detection', 0, 255, nothing)
cv2.createTrackbar('Threshold2', 'Canny Edge Detection', 255, 255, nothing)

while True:
    # Get trackbar positions
    threshold1 = cv2.getTrackbarPos('Threshold1', 'Canny Edge Detection')
    threshold2 = cv2.getTrackbarPos('Threshold2', 'Canny Edge Detection')

    # Apply Canny edge detection
    gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1, threshold2)

    # Display the edges
    cv2.imshow('Canny Edge Detection', edges)

    # Exit loop when 'q' key is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Function to do nothing on trackbar callback
def nothing(x):
    pass

# Function to calculate the angle of a line
def calculate_angle(line):
    x1, y1, x2, y2 = line[0]
    return np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

# Function to classify and group lines
def classify_and_group_lines(lines, gap_threshold):
    horizontal_lines = []
    vertical_lines = []

    # Classify lines into horizontal and vertical
    for line in lines:
        angle = calculate_angle(line)
        if abs(angle) < 45:  # Horizontal line
            horizontal_lines.append(line)
        else:  # Vertical line
            vertical_lines.append(line)

    # Function to group lines based on proximity
    def group_lines(lines, gap_threshold):
        lines = sorted(lines, key=lambda l: l[0][1] if abs(calculate_angle(l)) < 45 else l[0][0])
        grouped_lines = []
        current_group = [lines[0]]
        for line in lines[1:]:
            if abs((line[0][1] if abs(calculate_angle(line)) < 45 else line[0][0]) - (current_group[-1][0][1] if abs(calculate_angle(current_group[-1])) < 45 else current_group[-1][0][0])) <= gap_threshold:
                current_group.append(line)
            else:
                grouped_lines.append(current_group)
                current_group = [line]
        grouped_lines.append(current_group)
        return grouped_lines

    # Group horizontal and vertical lines
    grouped_horizontal_lines = group_lines(horizontal_lines, gap_threshold)
    grouped_vertical_lines = group_lines(vertical_lines, gap_threshold)

    return grouped_horizontal_lines, grouped_vertical_lines

# Function to draw the most extreme lines from each group
def draw_extreme_lines(image, grouped_lines, color=(0, 255, 0), thickness=2):
    for group in grouped_lines:
        x_coords = [line[0][0] for line in group] + [line[0][2] for line in group]
        y_coords = [line[0][1] for line in group] + [line[0][3] for line in group]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        # Draw line from the extreme points
        cv2.line(image, (x_min, y_min), (x_max, y_max), color, thickness)

# Load the image
image = cv2.imread('tic_tac_toe_board.jpg')

# Predefined ROI parameters
x, y, w, h = 642, 431, 737, 550

# Extract the ROI
roi_image = image[y:y+h, x:x+w]

# Apply Canny edge detection
threshold1, threshold2 = 0, 70
gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, threshold1, threshold2)

# Create a window for Hough Line Transform
cv2.namedWindow('Hough Line Transform')
cv2.createTrackbar('Threshold', 'Hough Line Transform', 100, 300, nothing)
cv2.createTrackbar('Min Line Length', 'Hough Line Transform', 50, 300, nothing)
cv2.createTrackbar('Max Line Gap', 'Hough Line Transform', 10, 100, nothing)
cv2.createTrackbar('Gap Threshold', 'Hough Line Transform', 100, 300, nothing)
cv2.createTrackbar('Apply Grouping (0/1)', 'Hough Line Transform', 1, 1, nothing)

while True:
    # Get trackbar positions
    threshold = cv2.getTrackbarPos('Threshold', 'Hough Line Transform')
    min_line_length = cv2.getTrackbarPos('Min Line Length', 'Hough Line Transform')
    max_line_gap = cv2.getTrackbarPos('Max Line Gap', 'Hough Line Transform')
    gap_threshold = cv2.getTrackbarPos('Gap Threshold', 'Hough Line Transform')
    apply_grouping = cv2.getTrackbarPos('Apply Grouping (0/1)', 'Hough Line Transform')

    # Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    # Create a copy of the ROI image to draw lines
    line_image = np.copy(roi_image)

    if lines is not None:
        if apply_grouping:
            grouped_horizontal_lines, grouped_vertical_lines = classify_and_group_lines(lines, gap_threshold)
            line_count = len(grouped_horizontal_lines) + len(grouped_vertical_lines)
            
            # Draw the most extreme lines from each group
            draw_extreme_lines(line_image, grouped_horizontal_lines)
            draw_extreme_lines(line_image, grouped_vertical_lines)
        else:
            line_count = len(lines)
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
        line_count = 0

    # Combine original ROI and line image
    combined_image = cv2.addWeighted(roi_image, 0.8, line_image, 1, 0)

    # Display the number of lines detected
    cv2.putText(combined_image, f'Lines detected: {line_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the lines on top of the original ROI
    cv2.imshow('Hough Line Transform', combined_image)

    # Exit loop when 'q' key is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np

# Function to calculate the angle of a line
def calculate_angle(line):
    x1, y1, x2, y2 = line[0]
    return np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

# Function to classify and group lines
def classify_and_group_lines(lines, gap_threshold):
    horizontal_lines = []
    vertical_lines = []

    # Classify lines into horizontal and vertical
    for line in lines:
        angle = calculate_angle(line)
        if abs(angle) < 45:  # Horizontal line
            horizontal_lines.append(line)
        else:  # Vertical line
            vertical_lines.append(line)

    # Function to group lines based on proximity
    def group_lines(lines, gap_threshold):
        lines = sorted(lines, key=lambda l: l[0][1] if abs(calculate_angle(l)) < 45 else l[0][0])
        grouped_lines = []
        current_group = [lines[0]]
        for line in lines[1:]:
            if abs((line[0][1] if abs(calculate_angle(line)) < 45 else line[0][0]) - (current_group[-1][0][1] if abs(calculate_angle(current_group[-1])) < 45 else current_group[-1][0][0])) <= gap_threshold:
                current_group.append(line)
            else:
                grouped_lines.append(current_group)
                current_group = [line]
        grouped_lines.append(current_group)
        return grouped_lines

    # Group horizontal and vertical lines
    grouped_horizontal_lines = group_lines(horizontal_lines, gap_threshold)
    grouped_vertical_lines = group_lines(vertical_lines, gap_threshold)

    return grouped_horizontal_lines, grouped_vertical_lines

# Function to draw the most extreme lines from each group
def draw_extreme_lines(image, grouped_lines, color=(0, 255, 0), thickness=2):
    for group in grouped_lines:
        x_coords = [line[0][0] for line in group] + [line[0][2] for line in group]
        y_coords = [line[0][1] for line in group] + [line[0][3] for line in group]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        # Draw line from the extreme points
        cv2.line(image, (x_min, y_min), (x_max, y_max), color, thickness)

# Load the image
image = cv2.imread('tic_tac_toe_board.jpg')

# Predefined ROI parameters
x, y, w, h = 642, 431, 737, 550

# Extract the ROI
roi_image = image[y:y+h, x:x+w]

# Apply Canny edge detection
threshold1, threshold2 = 0, 70
gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, threshold1, threshold2)

# Hough Line Transform parameters
threshold, min_length, max_gap = 6, 7, 2

# Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, minLineLength=min_length, maxLineGap=max_gap)

# Create a copy of the original image to draw lines on
line_image = np.copy(image)

if lines is not None:
    gap_threshold = 65
    grouped_horizontal_lines, grouped_vertical_lines = classify_and_group_lines(lines, gap_threshold)
    line_count = len(grouped_horizontal_lines) + len(grouped_vertical_lines)
    
    # Draw the most extreme lines from each group on the original image
    draw_extreme_lines(line_image[y:y+h, x:x+w], grouped_horizontal_lines)
    draw_extreme_lines(line_image[y:y+h, x:x+w], grouped_vertical_lines)
else:
    line_count = 0

# Display the number of lines detected
cv2.putText(line_image, f'Lines detected: {line_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Display the image with detected lines
cv2.imshow('Detected Lines', line_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np

# Function to calculate the angle of a line
def calculate_angle(line):
    x1, y1, x2, y2 = line[0]
    return np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

# Function to classify and group lines
def classify_and_group_lines(lines, gap_threshold):
    horizontal_lines = []
    vertical_lines = []

    # Classify lines into horizontal and vertical
    for line in lines:
        angle = calculate_angle(line)
        if abs(angle) < 45:  # Horizontal line
            horizontal_lines.append(line)
        else:  # Vertical line
            vertical_lines.append(line)

    # Function to group lines based on proximity
    def group_lines(lines, gap_threshold):
        lines = sorted(lines, key=lambda l: l[0][1] if abs(calculate_angle(l)) < 45 else l[0][0])
        grouped_lines = []
        current_group = [lines[0]]
        for line in lines[1:]:
            if abs((line[0][1] if abs(calculate_angle(line)) < 45 else line[0][0]) - (current_group[-1][0][1] if abs(calculate_angle(current_group[-1])) < 45 else current_group[-1][0][0])) <= gap_threshold:
                current_group.append(line)
            else:
                grouped_lines.append(current_group)
                current_group = [line]
        grouped_lines.append(current_group)
        return grouped_lines

    # Group horizontal and vertical lines
    grouped_horizontal_lines = group_lines(horizontal_lines, gap_threshold)
    grouped_vertical_lines = group_lines(vertical_lines, gap_threshold)

    return grouped_horizontal_lines, grouped_vertical_lines

# Function to draw the most extreme lines from each group
def draw_extreme_lines(image, grouped_lines, color=(0, 255, 0), thickness=2):
    for group in grouped_lines:
        x_coords = [line[0][0] for line in group] + [line[0][2] for line in group]
        y_coords = [line[0][1] for line in group] + [line[0][3] for line in group]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        # Draw line from the extreme points
        cv2.line(image, (x_min, y_min), (x_max, y_max), color, thickness)

# Predefined ROI parameters
x, y, w, h = 642, 431, 737, 550

# Apply Canny edge detection
threshold1, threshold2 = 0, 70

# Hough Line Transform parameters
threshold, min_length, max_gap = 6, 7, 2

# Grouping gap threshold
gap_threshold = 65

desired_width = 1920
desired_height = 1080

# Open the camera stream
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, or replace with the correct camera index

# Set the resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Check if the ROI is within the frame bounds
    if y + h <= frame.shape[0] and x + w <= frame.shape[1]:
        # Extract the ROI
        roi_image = frame[y:y+h, x:x+w]

        # Apply Canny edge detection
        gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, threshold1, threshold2)

        # Hough Line Transform
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, minLineLength=min_length, maxLineGap=max_gap)

        # Create a copy of the original frame to draw lines on
        line_image = np.copy(frame)

        if lines is not None:
            grouped_horizontal_lines, grouped_vertical_lines = classify_and_group_lines(lines, gap_threshold)
            line_count = len(grouped_horizontal_lines) + len(grouped_vertical_lines)
            
            # Draw the most extreme lines from each group on the original frame
            draw_extreme_lines(line_image[y:y+h, x:x+w], grouped_horizontal_lines)
            draw_extreme_lines(line_image[y:y+h, x:x+w], grouped_vertical_lines)
        else:
            line_count = 0

        # Display the number of lines detected
        cv2.putText(line_image, f'Lines detected: {line_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the frame with detected lines
        cv2.imshow('Detected Lines', line_image)
    else:
        print("ROI is out of frame bounds")

    # Exit loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2

# Desired resolution (replace with your image resolution)
desired_width = 1920
desired_height = 1080

# Open the camera stream
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, or replace with the correct camera index

# Set the resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Display the frame
    cv2.imshow('Camera Stream', frame)

    # Exit loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
